# Build Conversational Agents with the Python SDK

### [GENAI113](https://partner.cloudskillsboost.google/paths/523/course_templates/1316/labs/548755)


## Objective
In this lab, you will learn how to do the following:

- Set up a Colab Enterprise notebook with necessary packages.
- Create and configure a Conversational agent with generative settings.
- Develop a weather-fetching tool using OpenAPI and deploy it via Cloud Run.
- Integrate tools into agents and chain them for complex workflows.
- Implement multi-turn sessions for dynamic interactions.

## Task 1. Create a Colab Enterprise Notebook
In this section, you will set up a Colab Enterprise notebook environment in the Google Cloud Console.

1. In the Google Cloud Console, navigate to Vertex AI > Colab Enterprise.

2. In the My notebooks section, select the us-east1 region from the Region drop-down menu, if it is not already selected, and click the + New notebook button to create a new notebook.

3. Enter the following code snippet in the first cell and run it by pressing Shift + Return or Shift + Enter

In [1]:
!pip install dfcx-scrapi

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.1/263.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.6 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e095c657fda3e8caf2140de737f38371d91fdfc62f58f181b3762a852f4ac8ba
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


Once cell execution is done, click on RESTART SESSION.

4. Paste the following code snippet into a new cell to authenticate Colab for calling Client Endpoints.

In [1]:
import sys

# Colab Auth needed to call Client Endpoints (i.e. vertexai)
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

## Task 2. Build a New Agent Application
In this section, you will create a blank Agent Application, which serves as the fundamental building block for chaining together Agents, Tools, and adding few-shot examples.

1. Run the following code snippet to set the PROJECT_ID variable value.

In [2]:
PROJECT_ID = "qwiklabs-gcp-00-3304b06953f9"

2. Run the following code snippet to create the agent named Cymbal Agents and note down the value of AGENT NAME.

In [3]:
from dfcx_scrapi.core.agents import Agents

a = Agents()
agent = a.create_agent(
    project_id=PROJECT_ID,
    display_name="Cymbal Agents",
    gcp_region="us-east1",
    playbook_agent=True
)

panel = "(playbooks/00000000-0000-0000-0000-000000000000/basics//right-panel:simulator)"
print(f"AGENT LINK: https://vertexaiconversation.cloud.google.com/{agent.name}/{panel}")
print(f"AGENT NAME: {agent.name}")

AGENT LINK: https://vertexaiconversation.cloud.google.com/projects/qwiklabs-gcp-00-3304b06953f9/locations/us-east1/agents/3748b70e-c055-4f99-bf63-7b09ca60d06a/(playbooks/00000000-0000-0000-0000-000000000000/basics//right-panel:simulator)
AGENT NAME: projects/qwiklabs-gcp-00-3304b06953f9/locations/us-east1/agents/3748b70e-c055-4f99-bf63-7b09ca60d06a


3. Run the following code snippet to update the generative settings for the Conversational agent.

*Note:* Replace the placeholder YOUR_AGENT_NAME in the following code snippet with the actual Agent Name generated in the previous output.

In [6]:
import requests
from google.oauth2 import id_token
from google.auth.transport.requests import Request
import google.auth
import json

project_id = "qwiklabs-gcp-00-3304b06953f9"
location_id = "us-east1"

#Replace your Agent Name
agent_name = "projects/qwiklabs-gcp-00-3304b06953f9/locations/us-east1/agents/3748b70e-c055-4f99-bf63-7b09ca60d06a"
endpoint_base = "https://us-east1-dialogflow.googleapis.com"

def get_access_token():
    """Gets an access token using google-auth library."""
    creds, project = google.auth.default()
    auth_req = Request()
    creds.refresh(auth_req)
    return creds.token

def update_generative_settings(new_model=None, new_language_code=None):
    """Updates the generative settings for the Conversational agent."""
    access_token = get_access_token()
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }
    generative_settings_name = f"{agent_name}/generativeSettings"
    url = f"{endpoint_base}/v3/{generative_settings_name}"
    payload = {}
    update_mask_fields = []

    if new_model is not None:
        payload["llmModelSettings"] = {"model": new_model}
        update_mask_fields.append("llmModelSettings.model")

    if new_language_code is not None:
        payload["languageCode"] = new_language_code
        update_mask_fields.append("languageCode")

    if not payload:
        print("No settings to update.")
        return

    params = {"updateMask": ",".join(update_mask_fields)}

    try:
        response = requests.patch(url, headers=headers, params=params, json=payload)
        response.raise_for_status()  # Raise an exception for bad status codes
        updated_settings = response.json()
        print("Generative settings updated successfully:")
        print(json.dumps(updated_settings, indent=2))
        return updated_settings
    except requests.exceptions.RequestException as e:
        print(f"Error updating generative settings: {e}")
        if response is not None:
            print(f"Response status code: {response.status_code}")
            print(f"Response body: {response.text}")
        return None

# Example usage:
update_generative_settings(new_model="gemini-2.0-flash-001", new_language_code="en")

Generative settings updated successfully:
{
  "languageCode": "en",
  "name": "projects/qwiklabs-gcp-00-3304b06953f9/locations/us-east1/agents/3748b70e-c055-4f99-bf63-7b09ca60d06a/generativeSettings",
  "llmModelSettings": {
    "model": "gemini-2.0-flash-001"
  }
}


{'languageCode': 'en',
 'name': 'projects/qwiklabs-gcp-00-3304b06953f9/locations/us-east1/agents/3748b70e-c055-4f99-bf63-7b09ca60d06a/generativeSettings',
 'llmModelSettings': {'model': 'gemini-2.0-flash-001'}}

### Test Your Agent
When your Agent application is first created, there is a Default Generative Playbook created, which has no goal or instructions. However, you can still interact with it like a generic LLM model.

One of the great aspects of AI Applications is that it automatically handles session management for you. All Agent Applications are immediately production-scalable and ready, because they are backed by production-grade, scalable Google infrastructure.

Want to open your app to 100,000 users immediately? We've got you covered! And no extra work on your end.

4. Run the following code snippet to generate a session id for your agent.

In [7]:
from dfcx_scrapi.core.sessions import Sessions
s = Sessions()
session_id = s.build_session_id(agent.name)
session_id

'projects/qwiklabs-gcp-00-3304b06953f9/locations/us-east1/agents/3748b70e-c055-4f99-bf63-7b09ca60d06a/sessions/cad197b2-e380-4713-88f5-07192d366d8e'

5. Now, use the session id, along with the agent's name and a user prompt, to detect the intent.

In [9]:
res = s.detect_intent(agent.name, session_id, "hey, how are you?")
s.parse_result(res)

<font color='green'><b> USER QUERY:</font></b> hey, how are you?

<font color='green'><b>AGENT RESPONSE:</font></b> I am doing well, thank you for asking. How can I help you today?


6. Use a different user query to test the agent response.

In [10]:
 res = s.detect_intent(agent.name, session_id, "what kind of models do you have?")
 s.parse_result(res)

<font color='green'><b> USER QUERY:</font></b> what kind of models do you have?

<font color='green'><b>AGENT RESPONSE:</font></b> I do not have access to information about the specific models I use. I am a large language model, and I can assist you with various tasks. How can I help you today?


## Task 3. Update the Default Playbook
Our Agent isn't very useful at this stage, so let's update the goal and instructions to do something more worthwhile. We'll first fetch the handy playbooks_map so we can easily reference IDs that are required by the API endpoints.

1. To fetch the playbooks_map run the following code snippet.

In [11]:
from dfcx_scrapi.core.playbooks import Playbooks

p = Playbooks(agent.name)

playbooks_map = p.get_playbooks_map(agent.name, reverse=True)
playbooks_map

{'Default Generative Playbook': 'projects/qwiklabs-gcp-00-3304b06953f9/locations/us-east1/agents/3748b70e-c055-4f99-bf63-7b09ca60d06a/playbooks/00000000-0000-0000-0000-000000000000'}

2. Running the code snippet below will show that no goals or instruction sets have been defined yet.

In [13]:
playbook = p.get_playbook(playbooks_map["Default Generative Playbook"])
print(f"GOAL: {playbook.goal}")
print(f"INSTRUCTIONS: {playbook.instruction}")

GOAL: Default goal
INSTRUCTIONS: 


3. Run the below code snippet to provide a simple goal and instruction set to get started.

In [14]:
playbook = p.update_playbook(
    playbooks_map["Default Generative Playbook"],
    goal="You are a friendly Tesla service center agent.\nYour job is to help users book appointments and answer any questions they have.",
    instructions=["Greet the user.", "Answer any questions the have to the best of your ability."]
    )

print(f"GOAL: {playbook.goal}")
print(f"INSTRUCTIONS: {playbook.instruction}")

GOAL: You are a friendly Tesla service center agent.
Your job is to help users book appointments and answer any questions they have.
INSTRUCTIONS: steps {
  text: "Greet the user."
}
steps {
  text: "Answer any questions the have to the best of your ability."
}



### Test Agent Again

4. Now that we've updated the goal and instruction set, let's see how our Agent application reacts.

In [16]:
from dfcx_scrapi.core.sessions import Sessions

s = Sessions()

session_id = s.build_session_id(agent.name)
res = s.detect_intent(agent.name, session_id, "what kind of models do you have?")
s.parse_result(res)

<font color='green'><b> USER QUERY:</font></b> what kind of models do you have?

<font color='green'><b>AGENT RESPONSE:</font></b> I am sorry, I do not have the ability to provide information on Tesla models. I can only help with booking appointments.


## Task 4. Create Tools
Tools allow your Agent to interact with the outside world.

There are 3 primary tool types that you can use:

- OpenAPI Spec
- Functions
- Data Stores

In this section, we'll create an OpenAPI Spec tool that can do the following:

- Find the weather using the National Weather Service Web API.

For more information, see Vertex Agents Tools.

### Deploy Cloud Run Endpoints
Using the following code snippets, deploy a Cloud Run Endpoint to call the National Weather Service API.

1. Create the get_weather directory and navigate to it.

In [17]:
%mkdir get_weather
%cd get_weather

/content/get_weather


2. Create the file named requirements.txt and define required dependencies in to it

In [18]:
%%writefile requirements.txt
functions-framework==3.*
firebase-admin==6.2.*
firebase-functions
requests==2.*

Writing requirements.txt


3. Run the following code snippet to create the main.py file to call the National Weather Service API and use the get_weather directory configurations to create the Cloud Run named get_weather_fun.

In [20]:
!wget https://raw.githubusercontent.com/GoogleCloudPlatform/dfcx-scrapi/main/data/get_weather_tool.py
!mv get_weather_tool.py main.py
!gcloud config set project $PROJECT_ID
!gcloud functions deploy get_weather_fun --region "us-east1" --runtime python311 --trigger-http --entry-point main --no-allow-unauthenticated --gen2

--2025-07-12 22:20:58--  https://raw.githubusercontent.com/GoogleCloudPlatform/dfcx-scrapi/main/data/get_weather_tool.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1863 (1.8K) [text/plain]
Saving to: ‘get_weather_tool.py’

get_weather_tool.py 100%[===================>]   1.82K  --.-KB/s    in 0s      

2025-07-12 22:20:58 (25.8 MB/s) - ‘get_weather_tool.py’ saved [1863/1863]

Updated property [core/project].
You can view your function in the Cloud Console here: https://console.cloud.google.com/functions/details/us-east1/get_weather_fun?project=qwiklabs-gcp-00-3304b06953f9

buildConfig:
  automaticUpdatePolicy: {}
  build: projects/477457719226/locations/us-east1/builds/42e63455-c7d7-43c5-b0a1-7523910a5226
  dockerRegistry: ARTIFACT_REGISTRY
  dockerRepo

Note down the URI for your deployed Cloud Run, highlighted in the output image above. You can also view the URI in the Cloud Run console.

### Define OpenAPI Specs
4. Run the following code snippet to define the OpenAPI specification with the Cloud Run Endpoint. Before proceeding, make sure you replace the URL in the get_weather_spec with the CLOUD RUN ENDPOINT that you just deployed!

**Note:** Be sure to replace YOUR_CLOUD_RUN_ENDPOINT with the URI for your Cloud Run service, as mentioned in the last step.

In [21]:
get_weather_spec = """
openapi: 3.0.0
info:
  title: get_weather
  version: 1.0.0

servers:
  - url: https://get-weather-fun-tmddkae5ja-ue.a.run.app

paths:
  /get_weather_grid:
    get:
      summary: Returns the current grid information for a city and state
      operationId: get_weather_grid
      parameters:
        - name: latitude
          in: query
          required: true
          schema:
            type: string
        - name: longitude
          in: query
          required: true
          schema:
            type: string
      responses:
        '200':
          description: OK
          content:
            application/json:
              schema:
                type: object
                properties:
                  data:
                    type: string
"""

### Call Create Tools
For creating the OpenAPI Spec tool, we'll use the build_open_api_tool helper method to create the proper Tool object, then pass it to the create_tool method and capture the result.

5. Run the following code snippet to create the tool.

In [22]:
from dfcx_scrapi.core.tools import Tools

t = Tools()

# Tool 1 - get_weather
weather_tool = t.build_open_api_tool(
    "get_weather_fun",
    spec=get_weather_spec,
    description="Get the current weather for the provided city."
    )
tool1 = t.create_tool(agent.name, weather_tool)
print(str(tool1).lower())


name: "projects/qwiklabs-gcp-00-3304b06953f9/locations/us-east1/agents/3748b70e-c055-4f99-bf63-7b09ca60d06a/tools/57c16e34-f020-4730-a592-5cf2c7105cfd"
display_name: "get_weather_fun"
description: "get the current weather for the provided city."
open_api_spec {
  text_schema: "\nopenapi: 3.0.0\ninfo:\n  title: get_weather\n  version: 1.0.0\n\nservers:\n  - url: https://get-weather-fun-tmddkae5ja-ue.a.run.app\n\npaths:\n  /get_weather_grid:\n    get:\n      summary: returns the current grid information for a city and state\n      operationid: get_weather_grid\n      parameters:\n        - name: latitude\n          in: query\n          required: true\n          schema:\n            type: string\n        - name: longitude\n          in: query\n          required: true\n          schema:\n            type: string\n      responses:\n        \'200\':\n          description: ok\n          content:\n            application/json:\n              schema:\n                type: object\n           

## Task 5. Create Weather Agent
Now that we have one new tool named get_weather_fun to work with, let's apply them to a new Agent in our application.

In [23]:
 from dfcx_scrapi.core.playbooks import Playbooks

 p = Playbooks(agent_id=agent.name)

 instructions = [
     "Use the ${TOOL:get_weather_fun} to get the current city/state weather grid information.",
     "- If the user only provides the city you can assume the state unless otherwise specified."]

 # New Playbook
 weather_agent = p.create_playbook(
     agent.name,
     display_name="Weather Agent",
     referenced_tools=[tool1.name],
     goal="You are a senior weather advisor at a network broadcast station. Your job is to predict the weather!",
     instructions=instructions
 )

 playbooks_map = p.get_playbooks_map(agent.name, reverse=True)
 playbooks_map

{'Default Generative Playbook': 'projects/qwiklabs-gcp-00-3304b06953f9/locations/us-east1/agents/3748b70e-c055-4f99-bf63-7b09ca60d06a/playbooks/00000000-0000-0000-0000-000000000000',
 'Weather Agent': 'projects/qwiklabs-gcp-00-3304b06953f9/locations/us-east1/agents/3748b70e-c055-4f99-bf63-7b09ca60d06a/playbooks/87ca0af5-060c-42ed-b1af-8d5702b19756'}

## Task 6. Chain Agents and Multi Turn Session
In order for our Agents to work together properly, we need to chain them together and update the instruction set.

Remember, our current architecture looks like this:

- Default Generative Agent
- Weather Agent

These are mutually exclusive Agents that cannot interact yet.

We want our architecture to look like this instead:

- Default Generative Playbook -> Weather Agent

1. In order to do this, let's update the instruction set on our Default Generative Playbook.

In [24]:
p.update_playbook(
    playbooks_map["Default Generative Playbook"],
    instructions=["If the user needs help with the weather, call ${PLAYBOOK: Weather Agent}"]
)

name: "projects/qwiklabs-gcp-00-3304b06953f9/locations/us-east1/agents/3748b70e-c055-4f99-bf63-7b09ca60d06a/playbooks/00000000-0000-0000-0000-000000000000"
display_name: "Default Generative Playbook"
goal: "You are a friendly Tesla service center agent.\nYour job is to help users book appointments and answer any questions they have."
token_count: 93
referenced_playbooks: "projects/qwiklabs-gcp-00-3304b06953f9/locations/us-east1/agents/3748b70e-c055-4f99-bf63-7b09ca60d06a/playbooks/87ca0af5-060c-42ed-b1af-8d5702b19756"
instruction {
  steps {
    text: "Greet the user."
  }
  steps {
    text: "Answer any questions the have to the best of your ability."
  }
  steps {
    text: "If the user needs help with the weather, call ${PLAYBOOK:Weather Agent}"
  }
}

### Test Agent Again
2. With our Agents chained together, let's test with a new query about weather in a specific city!

In [25]:
from dfcx_scrapi.core.sessions import Sessions

s = Sessions()

session_id = s.build_session_id(agent.name)
res = s.detect_intent(agent.name, session_id, "what is the weather like in Austin, Texas?")
s.parse_result(res)


<font color='green'><b> USER QUERY:</font></b> what is the weather like in Austin, Texas?

<font color='green'><b>AGENT RESPONSE:</font></b> Okay, I'm ready to provide a weather forecast for Austin, Texas. I will use the available tools to get the current weather grid information.


### Multi Turn Session
3. Example of a Multi Turn Session with some cleaner readability for colab testing.

In [26]:
session_id = s.build_session_id(agent.name)

conversation = [
    "What colors does the model 3 come in?",
    "What's the weather like in Boston right now? I'm thinking about walking to the Tesla dealership."
    ]

i = 1
for utterance in conversation:
  print(f"\n--- TURN {i} --- \n")
  res = s.detect_intent(agent.name, session_id, utterance)
  s.parse_result(res)
  i+=1


--- TURN 1 --- 



<font color='green'><b> USER QUERY:</font></b> What colors does the model 3 come in?

<font color='green'><b>AGENT RESPONSE:</font></b> I do not have access to the Model 3 color options. I am here to help with booking appointments or answering questions about your existing Tesla services. Is there anything else I can help you with?



--- TURN 2 --- 



<font color='green'><b> USER QUERY:</font></b> What's the weather like in Boston right now? I'm thinking about walking to the Tesla dealership.

<font color='green'><b>AGENT RESPONSE:</font></b> I can help with that! I will need a latitude and longitude for Boston. Would you like me to look that up?
